In [2]:
import sys
import simulator as sim
import numpy as np
import sklearn as skl
from simulator import (
    Simulator, string_to_micro, micro_to_time,
    BUY, SELL, SHORT, EXCH_INET,
    BOOK_DEPTH1_PRICE, ORDER_EVENTS,ORDER_EXECUTED
    )
import simulator as sim

class HKAlgo(object):
    def __init__(self, session, date, ticker, start_time, end_time):
        # Save session information
        self.session = session
        self.date = date
        self.ticker = ticker[0]
        self.start_time = start_time
        self.end_time = end_time
        self.order_size = 1
        
        # TODO: Additional data structures
        self.book = {"ask": [], "bid": []}
        
        # Suscribe to tickers and set first timer
        self.session.add_timer(self.start_time, self.timer_OBS_callback)
        self.session.subscribe_ticker_all_feeds(self.ticker)
 
    def update_order(self, ticker):
        self.post_order(ticker)

    def timer_OBS_callback(self, time):
        self.update_order(self.ticker)
         
    # def cancel_order:
    
    # def stop_loss:

    def post_order(self, ticker):
        bid, ask = self.session.get_inside_market(self.ticker)
        b_price = bid['price'] # = bestBidPrice()
        a_price = ask['price'] # = bestAskPrice()
        # Prices not defined generating errors
        print b_price
        print a_price
        orders = self.session.get_orders_by_ticker(ticker)

        if not orders:
            self.session.add_order(ticker, BUY, self.order_size, b_price, exchange=EXCH_INET)
            self.session.add_order(ticker, SELL, self.order_size, a_price, exchange=EXCH_INET) 
            self.session.subscribe_event(ticker, sim.BOOK_DEPTH1_PRICE + sim.ORDER_EXECUTED, self.hit_ask_callback)
            self.session.subscribe_event(ticker, sim.BOOK_DEPTH1_PRICE + sim.ORDER_EXECUTED, self.hit_bid_callback)
            print "Posted orders"

    def hit_ask_callback(self, ticker, event_params):
        if "executed_orders" in event_params:
            time = self.session.current_time()
            for executed_order in event_params["executed_orders"]:
                order = executed_order["order"]
                side = order["side"]
                if side == "B":
                    self.book["ask"] = [time]
                    print "Yeah man I'm buying, it's now {0}!".format(micro_to_time(time))
        
    def hit_bid_callback(self, ticker, event_params):
        if "executed_orders" in event_params:
            time = self.session.current_time()
            for executed_order in event_params["executed_orders"]:
                order = executed_order["order"]
                side = order["side"]
                if side == "S":
                    self.book["ask"] = [time]
                    print "Yeah man I'm selling, it's now {0}!".format(micro_to_time(time))
    
    
    def end(self):
        print "Done"
        return
       
    
date = "20150121"
SNP500_tickers = ["ABT", "ABBV", "ACN", "ACE", "ADBE", "ADT", "AAP", "AES", "AET", "AFL", "AMG", "A", "GAS", 
                  "APD", "ARG", "AKAM", "AA", "AGN", "ALXN", "ALLE", "ADS", "ALL", "ALTR", "MO", "AMZN", "AEE", 
                  "AAL", "AEP", "AXP", "AIG", "AMT", "AMP", "ABC", "AME", "AMGN", "APH", "APC", "ADI", "AON", 
                  "APA", "AIV", "AMAT", "ADM", "AIZ", "T", "ADSK", "ADP", "AN", "AZO", "AVGO", "AVB", "AVY", 
                  "BHI", "BLL", "BAC", "BK", "BCR", "BXLT", "BAX", "BBT", "BDX", "BBBY", "BRK-B", "BBY", "BLX", 
                  "HRB", "BA", "BWA", "BXP", "BSK", "BMY", "BRCM", "BF-B", "CHRW", "CA", "CVC", "COG", "CAM", 
                  "CPB", "COF", "CAH", "HSIC", "KMX", "CCL", "CAT", "CBG", "CBS", "CELG", "CNP", "CTL", "CERN", 
                  "CF", "SCHW", "CHK", "CVX", "CMG", "CB", "CI", "XEC", "CINF", "CTAS", "CSCO", "C", "CTXS", 
                  "CLX", "CME", "CMS", "COH", "KO", "CCE", "CTSH", "CL", "CMCSA", "CMA", "CSC", "CAG", "COP", 
                  "CNX", "ED", "STZ", "GLW", "COST", "CCI", "CSX", "CMI", "CVS", "DHI", "DHR", "DRI", "DVA", 
                  "DE", "DLPH", "DAL", "XRAY", "DVN", "DO", "DTV", "DFS", "DISCA", "DISCK", "DG", "DLTR", "D", 
                  "DOV", "DOW", "DPS", "DTE", "DD", "DUK", "DNB", "ETFC", "EMN", "ETN", "EBAY", "ECL", "EIX", 
                  "EW", "EA", "EMC", "EMR", "ENDP", "ESV", "ETR", "EOG", "EQT", "EFX", "EQIX", "EQR", "ESS", 
                  "EL", "ES", "EXC", "EXPE", "EXPD", "ESRX", "XOM", "FFIV", "FB", "FAST", "FDX", "FIS", "FITB", 
                  "FSLR", "FE", "FSIV", "FLIR", "FLS", "FLR", "FMC", "FTI", "F", "FOSL", "BEN", "FCX", "FTR", 
                  "GME", "GPS", "GRMN", "GD", "GE", "GGP", "GIS", "GM", "GPC", "GNW", "GILD", "GS", "GT", "GOOGL", 
                  "GOOG", "GWW", "HAL", "HBI", "HOG", "HAR", "HRS", "HIG", "HAS", "HCA", "HCP", "HCN", "HP", 
                  "HES", "HPQ", "HD", "HON", "HRL", "HSP", "HST", "HCBK", "HUM", "HBAN", "ITW", "IR", "INTC", 
                  "ICE", "IBM", "IP", "IPG", "IFF", "INTU", "ISRG", "IVZ", "IRM", "JEC", "JBHT", "JNJ", "JCI", 
                  "JOY", "JPM", "JNPR", "KSU", "K", "KEY", "GMCR", "KMB", "KIM", "KMI", "KLAC", "KSS", "KRFT", 
                  "KR", "LB", "LLL", "LH", "LRCX", "LM", "LEG", "LEN", "LVLT", "LUK", "LLY", "LNC", "LLTC", "LMT", 
                  "L", "LOW", "LYB", "MTB", "MAC", "M", "MNK", "MRO", "MPC", "MAR", "MMC", "MLM", "MAS", "MA", 
                  "MAT", "MKC", "MCD", "MHFI", "MCK", "MJN", "MMV", "MDT", "MRK", "MET", "KORS", "MCHP", "MU", 
                  "MSFT", "MHK", "TAP", "MDLZ", "MON", "MNST", "MCO", "MS", "MOS", "MSI", "MUR", "MYL", "NDAQ", 
                  "NOV", "NAVI", "NTAP", "NFLX", "NWL", "NFX", "NEM", "NWSA", "NEE", "NLSN", "NKE", "NI", "NE", 
                  "NBL", "JWN", "NSC", "NTRS", "NOC", "NRG", "NUE", "NVDA", "ORLY", "OXY", "OMC", "OKE", "ORCL", 
                  "OI", "PCAR", "PLL", "PH", "PDCO", "PAYX", "PNR", "PBCT", "POM", "PEP", "PKI", "PRGO", "PFE", 
                  "PCG", "PM", "PSX", "PNW", "PXD", "PBI", "PCL", "PNC", "RL", "PPG", "PPL", "PX", "PCP", "PCLN", 
                  "PFG", "PG", "PGR", "PLD", "PRU", "PEG", "PSA", "PHM", "PVH", "QRVO", "PWR", "QCOM", "DGX", 
                  "RRC", "RTN", "O", "RHT", "REGN", "RF", "RSG", "RAI", "RHI", "ROK", "COL", "ROP", "ROST", "RLC", 
                  "R", "CRM", "SNDK", "SCG", "SLB", "SNI", "STX", "SEE", "SRE", "SHW", "SIAL", "SPG", "SWKS", 
                  "SLG", "SJM", "SNA", "SO", "LUV", "SWN", "SE", "STJ", "SWK", "SPLS", "SBUX", "HOT", "STT", 
                  "SRCL", "SYK", "STI", "SYMC", "SYY", "TROW", "TGT", "TEL", "TE", "TGNA", "THC", "TDC", "TSO", 
                  "TXN", "TXT", "HSY", "TRV", "TMO", "TIF", "TWX", "TWC", "TJK", "TMK", "TSS", "TSCO", "RIG", 
                  "TRIP", "FOXA", "TSN", "TYC", "UA", "UNP", "UNH", "UPS", "URI", "UTX", "UHS", "UNM", "URBN", 
                  "VFC", "VLO", "VAR", "VTR", "VRSN", "VZ", "VRTX", "VIAB", "V", "VNO", "VMC", "WMT", "WBA", 
                  "DIS", "WM", "WAT", "ANTM", "WFC", "WDC", "WU", "WY", "WHR", "WFM", "WMB", "WEC", "WYN", 
                  "WYNN", "XEL", "XRX", "XLNX", "XL", "XYL", "YHOO", "YUM", "ZBH", "ZION", "ZTS"]
start_time = sim.string_to_micro("9:30")
end_time = sim.string_to_micro("10:00")
simul = sim.Simulator(HKAlgo)
simul.run(date, ["IVV"], use_om=True, start_time=start_time, end_time=end_time)



ImportError: No module named simulator